In [1]:
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

### Preprocessing the data

In [2]:
from collections import defaultdict
from gensim import corpora

documents = [
    "Human machine interface for lab abc computer applications",
    "A survey of user opinion of computer system response time",
    "The EPS user interface management system",
    "System and human system engineering testing of EPS",
    "Relation of user perceived response time to error measurement",
    "The generation of random binary unordered trees",
    "The intersection graph of paths in trees",
    "Graph minors IV Widths of trees and well quasi ordering",
    "Graph minors A survey",
]

# remove common words and tokenize
stoplist = set('for a of the and to in'.split())
texts = [
    [word for word in document.lower().split() if word not in stoplist]
    for document in documents
]

# remove words that appear only once
frequency = defaultdict(int)
for text in texts:
    for token in text:
        frequency[token] += 1

texts = [
    [token for token in text if frequency[token] > 1]
    for text in texts
]

dictionary = corpora.Dictionary(texts)
corpus = [dictionary.doc2bow(text) for text in texts]

2022-07-15 12:15:12,374 : INFO : adding document #0 to Dictionary<0 unique tokens: []>
2022-07-15 12:15:12,375 : INFO : built Dictionary<12 unique tokens: ['computer', 'human', 'interface', 'response', 'survey']...> from 9 documents (total 29 corpus positions)
2022-07-15 12:15:12,375 : INFO : Dictionary lifecycle event {'msg': "built Dictionary<12 unique tokens: ['computer', 'human', 'interface', 'response', 'survey']...> from 9 documents (total 29 corpus positions)", 'datetime': '2022-07-15T12:15:12.375782', 'gensim': '4.2.0', 'python': '3.10.4 (main, Apr 26 2022, 19:36:29) [Clang 13.1.6 (clang-1316.0.21.2)]', 'platform': 'macOS-12.4-arm64-arm-64bit', 'event': 'created'}


In [3]:
from gensim import models

tfidf = models.TfidfModel(corpus)  # step 1 -- initialize a model


2022-07-15 12:15:22,305 : INFO : collecting document frequencies
2022-07-15 12:15:22,305 : INFO : PROGRESS: processing document #0
2022-07-15 12:15:22,305 : INFO : TfidfModel lifecycle event {'msg': 'calculated IDF weights for 9 documents and 12 features (28 matrix non-zeros)', 'datetime': '2022-07-15T12:15:22.305935', 'gensim': '4.2.0', 'python': '3.10.4 (main, Apr 26 2022, 19:36:29) [Clang 13.1.6 (clang-1316.0.21.2)]', 'platform': 'macOS-12.4-arm64-arm-64bit', 'event': 'initialize'}


In [4]:
doc_bow = [(0, 1), (1, 1)]
print(tfidf[doc_bow])  # step 2 -- use the model to transform vectors

[(0, 0.7071067811865476), (1, 0.7071067811865476)]


In [6]:
corpus_tfidf = tfidf[corpus]
for doc in corpus_tfidf:
    print(doc)

[(0, 0.5773502691896257), (1, 0.5773502691896257), (2, 0.5773502691896257)]
[(0, 0.44424552527467476), (3, 0.44424552527467476), (4, 0.44424552527467476), (5, 0.3244870206138555), (6, 0.44424552527467476), (7, 0.3244870206138555)]
[(2, 0.5710059809418182), (5, 0.4170757362022777), (7, 0.4170757362022777), (8, 0.5710059809418182)]
[(1, 0.49182558987264147), (5, 0.7184811607083769), (8, 0.49182558987264147)]
[(3, 0.6282580468670046), (6, 0.6282580468670046), (7, 0.45889394536615247)]
[(9, 1.0)]
[(9, 0.7071067811865475), (10, 0.7071067811865475)]
[(9, 0.5080429008916749), (10, 0.5080429008916749), (11, 0.695546419520037)]
[(4, 0.6282580468670046), (10, 0.45889394536615247), (11, 0.6282580468670046)]


In [10]:
lsi_model = models.LsiModel(corpus_tfidf, id2word=dictionary, num_topics=2)  # initialize an LSI transformation
corpus_lsi = lsi_model[corpus_tfidf]  # create a double wrapper over the original corpus: bow->tfidf->fold-in-lsi

2022-07-15 12:16:14,808 : INFO : using serial LSI version on this node
2022-07-15 12:16:14,818 : INFO : updating model with new documents
2022-07-15 12:16:14,820 : INFO : preparing a new chunk of documents
2022-07-15 12:16:14,821 : INFO : using 100 extra samples and 2 power iterations
2022-07-15 12:16:14,823 : INFO : 1st phase: constructing (12, 102) action matrix
2022-07-15 12:16:14,825 : INFO : orthonormalizing (12, 102) action matrix
2022-07-15 12:16:14,827 : INFO : 2nd phase: running dense svd on (12, 9) matrix
2022-07-15 12:16:14,830 : INFO : computing the final decomposition
2022-07-15 12:16:14,833 : INFO : keeping 2 factors (discarding 47.565% of energy spectrum)
2022-07-15 12:16:14,835 : INFO : processed documents up to #9
2022-07-15 12:16:14,837 : INFO : topic #0(1.594): 0.703*"trees" + 0.538*"graph" + 0.402*"minors" + 0.187*"survey" + 0.061*"system" + 0.060*"response" + 0.060*"time" + 0.058*"user" + 0.049*"computer" + 0.035*"interface"
2022-07-15 12:16:14,838 : INFO : topic #

In [9]:
lsi_model.print_topics(2)

2022-07-15 12:16:12,868 : INFO : topic #0(1.594): 0.703*"trees" + 0.538*"graph" + 0.402*"minors" + 0.187*"survey" + 0.061*"system" + 0.060*"time" + 0.060*"response" + 0.058*"user" + 0.049*"computer" + 0.035*"interface"
2022-07-15 12:16:12,871 : INFO : topic #1(1.476): -0.460*"system" + -0.373*"user" + -0.332*"eps" + -0.328*"interface" + -0.320*"response" + -0.320*"time" + -0.293*"computer" + -0.280*"human" + -0.171*"survey" + 0.161*"trees"


[(0,
  '0.703*"trees" + 0.538*"graph" + 0.402*"minors" + 0.187*"survey" + 0.061*"system" + 0.060*"time" + 0.060*"response" + 0.058*"user" + 0.049*"computer" + 0.035*"interface"'),
 (1,
  '-0.460*"system" + -0.373*"user" + -0.332*"eps" + -0.328*"interface" + -0.320*"response" + -0.320*"time" + -0.293*"computer" + -0.280*"human" + -0.171*"survey" + 0.161*"trees"')]

In [11]:
# both bow->tfidf and tfidf->lsi transformations are actually executed here, on the fly
for doc, as_text in zip(corpus_lsi, documents):
    print(doc, as_text)

[(0, 0.06600783396090452), (1, -0.5200703306361849)] Human machine interface for lab abc computer applications
[(0, 0.19667592859142735), (1, -0.7609563167700043)] A survey of user opinion of computer system response time
[(0, 0.08992639972446619), (1, -0.724186062675251)] The EPS user interface management system
[(0, 0.07585847652178304), (1, -0.6320551586003431)] System and human system engineering testing of EPS
[(0, 0.10150299184980348), (1, -0.5737308483002953)] Relation of user perceived response time to error measurement
[(0, 0.7032108939378309), (1, 0.1611518021402598)] The generation of random binary unordered trees
[(0, 0.8774787673119826), (1, 0.1675890686465962)] The intersection graph of paths in trees
[(0, 0.9098624686818573), (1, 0.1408655362871922)] Graph minors IV Widths of trees and well quasi ordering
[(0, 0.6165825350569281), (1, -0.05392907566389246)] Graph minors A survey


In [12]:
import os
import tempfile

with tempfile.NamedTemporaryFile(prefix='model-', suffix='.lsi', delete=False) as tmp:
    lsi_model.save(tmp.name)  # same for tfidf, lda, ...

loaded_lsi_model = models.LsiModel.load(tmp.name)

os.unlink(tmp.name)

2022-07-15 12:16:30,833 : INFO : Projection lifecycle event {'fname_or_handle': '/var/folders/1h/nl4lppwj1hgdkrq7c63vgdpr0000gn/T/model-f0eqohl0.lsi.projection', 'separately': 'None', 'sep_limit': 10485760, 'ignore': frozenset(), 'datetime': '2022-07-15T12:16:30.833210', 'gensim': '4.2.0', 'python': '3.10.4 (main, Apr 26 2022, 19:36:29) [Clang 13.1.6 (clang-1316.0.21.2)]', 'platform': 'macOS-12.4-arm64-arm-64bit', 'event': 'saving'}
2022-07-15 12:16:30,835 : INFO : saved /var/folders/1h/nl4lppwj1hgdkrq7c63vgdpr0000gn/T/model-f0eqohl0.lsi.projection
2022-07-15 12:16:30,835 : INFO : LsiModel lifecycle event {'fname_or_handle': '/var/folders/1h/nl4lppwj1hgdkrq7c63vgdpr0000gn/T/model-f0eqohl0.lsi', 'separately': 'None', 'sep_limit': 10485760, 'ignore': ['projection', 'dispatcher'], 'datetime': '2022-07-15T12:16:30.835773', 'gensim': '4.2.0', 'python': '3.10.4 (main, Apr 26 2022, 19:36:29) [Clang 13.1.6 (clang-1316.0.21.2)]', 'platform': 'macOS-12.4-arm64-arm-64bit', 'event': 'saving'}
2022